In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
%cd gdrive/MyDrive/anime_recommend_system/

/content/gdrive/MyDrive/anime_recommend_system


In [6]:
anime=pd.read_csv('anime.csv')
print(anime.shape)
anime.head()

(12294, 7)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
rating=pd.read_csv('rating.csv')
print(rating.shape)
rating.head()

(7813737, 3)


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [8]:
df=anime.merge(rating,on='anime_id')
print(df.shape)
df.head()

(7813727, 9)


,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,-1


In [9]:
df['user_rating']=df['rating_y']
df['rating']=df['rating_x']

In [10]:
# Dropping rating_x and rating_y column
df.drop(['rating_x','rating_y'],axis=1,inplace=True)

In [11]:
# Dropping all rows containing null values
df.dropna(axis=0,inplace=True)

In [12]:
# Grouping anime based on rating and seeing the no. of people rated a particular anime
df.groupby('name')['rating'].count().sort_values(ascending=False).head(10)

name
Death Note                          39340
Sword Art Online                    30583
Shingeki no Kyojin                  29584
Code Geass: Hangyaku no Lelouch     27718
Elfen Lied                          27506
Angel Beats!                        27183
Naruto                              25925
Fullmetal Alchemist                 25032
Fullmetal Alchemist: Brotherhood    24574
Toradora!                           24283
Name: rating, dtype: int64

In [13]:
df.groupby('name')['rating'].count().sort_values(ascending=False).reset_index().head(10)

,name,rating
0,Death Note,39340
1,Sword Art Online,30583
2,Shingeki no Kyojin,29584
3,Code Geass: Hangyaku no Lelouch,27718
4,Elfen Lied,27506
5,Angel Beats!,27183
6,Naruto,25925
7,Fullmetal Alchemist,25032
8,Fullmetal Alchemist: Brotherhood,24574
9,Toradora!,24283


In [14]:
# Now we are making a dataframe of above code by changing rating to totalRatingCount(number of users who rated a 
# particular movie)
anime_ratingcount = df.groupby('name')['rating'].count().reset_index().rename(columns = {'rating': 'totalRatingCount'})
anime_ratingcount.sample(7)

,name,totalRatingCount
10992,Yurumates wa?,71
3113,Gokujo.: Souda Onsen ni Ikou!!,80
8896,Shiokari Touge,1
3439,Happy Happy Clover,9
10015,Tokyo Fantasia,1
4387,Jungle Taitei Movie,5
4785,Kemeko Deluxe!,1156


In [15]:
# Now we are making a dataframe of above code by changing rating to totalRatingCount(number of users who rated a 
# particular movie)
anime_ratingcount = df.groupby('name')['rating'].count().reset_index().rename(columns = {'rating': 'totalRatingCount'})
anime_ratingcount.sample(7)

,name,totalRatingCount
7843,Pretty Rhythm: All Star Selection,12
10117,Tottoko Hamtarou OVA 4: Hamuchanzu no Mezase! ...,17
3968,Hyoutan Suzume,3
1194,Canaan,4526
1284,Chargeman Ken!,7
7157,Ojousama Sousamou,53
7831,Precure All Stars Movie DX3: Mirai ni Todoke! ...,105


In [16]:
# Merging df and anime_ratingcount dataset on name
df=df.merge(anime_ratingcount,on='name')
df.head()

,anime_id,name,genre,type,episodes,members,user_id,user_rating,rating,totalRatingCount
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,99,5,9.37,2199
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,152,10,9.37,2199
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,244,10,9.37,2199
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,271,10,9.37,2199
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,278,-1,9.37,2199


In [17]:
popularity_threshold = 50
rating_popular_anime= df.query('totalRatingCount >= @popularity_threshold')
print(rating_popular_anime.shape)
rating_popular_anime.head()      # only give rating of those movies for which atleast 50user have rated

(7748851, 10)


,anime_id,name,genre,type,episodes,members,user_id,user_rating,rating,totalRatingCount
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,99,5,9.37,2199
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,152,10,9.37,2199
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,244,10,9.37,2199
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,271,10,9.37,2199
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,200630,278,-1,9.37,2199


In [18]:
# Creating a Pivot matrix
anime_pivot=rating_popular_anime.pivot_table(index='name',columns='user_id',values='rating').fillna(0)
anime_pivot.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.54,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
.hack//G.U. Returner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
.hack//G.U. Trilogy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Now converting our pivot table to matrix using scipy.sparse library
from scipy.sparse import csr_matrix
anime_matrix = csr_matrix(anime_pivot.values)
# Applying NearestNeighbors algorithm
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(anime_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
anime_pivot.shape

(5651, 73502)

In [29]:
query_index = np.random.choice(anime_pivot.shape[0]) #shape[0] mean it will select row i.e any row index for a movie
print(query_index)   # query_index basically selecting a random movie on basis of which we are recommending
distances, indices = model_knn.kneighbors(anime_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

5645


In [30]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(anime_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, anime_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for s.CRY.ed:

1: Seihou Bukyou Outlaw Star, with distance of 0.6748813859688663:
2: Trigun, with distance of 0.6847282170131432:
3: The Big O, with distance of 0.6987419760566536:
4: Eureka Seven, with distance of 0.7105423776615094:
5: Trinity Blood, with distance of 0.7155507151637166:
